#Instalar bibliotecas y paquetes

In [ ]:
!pip install sentencepiece
!pip install torch
!pip install transformers
!pip install pytorch-lightning --upgrade
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00


#Importar modulos y bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pytorch_lightning as pl

from tqdm.auto import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
from google.colab import drive
# montar colab
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Rutas de los archivos TSV
ruta_tsv_train = '/content/drive/MyDrive/Datasets/x-final/es/translated_train.tsv'
ruta_tsv_test = '/content/drive/MyDrive/Datasets/x-final/es/test_2k.tsv'

# Leer los archivos TSV con pandas
df_train = pd.read_csv(ruta_tsv_train, sep='\t')
df_test = pd.read_csv(ruta_tsv_test, sep='\t')

# Rutas para guardar los archivos CSV
ruta_csv_train = '/content/drive/MyDrive/Datasets/x-final/es/translated_train.csv'
ruta_csv_test = '/content/drive/MyDrive/Datasets/x-final/es/test_2k.csv'

# Guardar los datos en formato CSV
df_train.to_csv(ruta_csv_train, index=False) #no se guardará la columna de índices del DataFrame en el archivo CSV
df_test.to_csv(ruta_csv_test, index=False) #no se guardará la columna de índices del DataFrame en el archivo CSV

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
df_train.head()

,id,sentence1,sentence2,label
0,1,"En París, en octubre de 1560, se reunió en sec...","En octubre de 1560, se reunió en secreto con e...",0
1,2,La temporada de la NBA de 1975: 76 fue la 30ª ...,La temporada 1975 - 76 de la Asociación Nacion...,1
2,3,"También hay discusiones específicas, debates d...","También hay discusiones públicas, discusiones ...",0
3,4,Cuando se pueden mantener tasas de flujo compa...,Los resultados son altos cuando se pueden mant...,1
4,5,Es la sede del distrito de Zerendi en la regió...,Es la sede del distrito de Zerendi en la regió...,1


In [ ]:
df_test.head()

,id,sentence1,sentence2,label
0,10,La excepción fue entre fines de 2005 y 2009 cu...,La excepción se dio entre fines del 2005 y 200...,1
1,12,El río Tabaci es una vertiente del río Leurda ...,El río Leurda es un afluente del río Tabaci en...,0
2,20,Jugó con los Kane County Cougars clase A en 19...,Jugó en 1993 con Portland Sea Dogs clase A y K...,0
3,26,"Winarsky es miembro de IEEE, Phi Beta Kappa, A...","Winarsky es miembro de ACM, IEEE, Phi Beta Kap...",1
4,27,En 1938 se convirtió en el antropólogo del gob...,En 1938 se convirtió en antropólogo del gobier...,0


In [ ]:
df_train.shape, df_test.shape

((49401, 4), (2000, 4))

#Cargando el modelo pre entrenado PEGASUS

In [ ]:
pegasus_model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(torch_device)
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
# Encontrar la longitud maxima de las cadenas
df_train["sentence1"].str.len().max(), df_train["sentence2"].str.len().max()

(259.0, 267.0)

In [ ]:
example =df_train.iloc[1]["sentence1"]
example

'La temporada de la NBA de 1975: 76 fue la 30ª temporada de la National Basketball Association.'

In [ ]:
example2 = df_train.iloc[1]["sentence2"]
example2

'La temporada 1975 - 76 de la Asociación Nacional de Baloncesto fue la temporada 30 de la NBA.'

In [ ]:
# Crear Dataset personalizado para PAW-X
class PawXDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=100, target_max_length=100):
        self.data = self.load_data(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length #limita la longitud de las secuencias de entrada
        self.target_max_length = target_max_length #limita la longitud de las secuencias de salida

    # Devuelve la longitud total de pares de oraciones en el archivo
    def __len__(self):
        return len(self.data)

    # Toma un índice como argumento y devuelve un elemento del conjunto de datos
    def __getitem__(self, index):
        item = self.data.iloc[index]
        sentence1 = item['sentence1']
        sentence2 = item['sentence2']

        # diccionario de la tokenizacion de la oracion de entrada
        inputs = self.tokenizer(
            sentence1, # La oración de entrada que se deseas tokenizar.
            max_length=self.max_length, # longitud máxima para las secuencias tokenizadas
            return_tensors='pt',
            padding='max_length', # se realiza el relleno hasta la longitud máxima especificada
            truncation=True # si excede la longitud máxima especificada
        )

        # diccionario de la tokenizacion de la oracion objetivo(salida)
        targets = self.tokenizer(
            sentence2, # La oración objetivo que se deseas tokenizar.
            max_length=self.target_max_length,
            return_tensors='pt',
            padding='max_length',
            truncation=True
        )

        return {
            # squeeze() se utiliza para eliminar las dimensiones adicionales agregadas durante la tokenización
            'input_ids': inputs['input_ids'].squeeze(), # Guarda los identificadores de tokens numéricos de la oración de entrada tokenizada
            'attention_mask': inputs['attention_mask'].squeeze(), # Guarda la máscara de atención asociada a la oración de entrada
            'target_ids': targets['input_ids'].squeeze(), # Guarda los identificadores de tokens numéricos de la oración objetivo tokenizada
            'target_attention_mask': targets['attention_mask'].squeeze(), # Guarda la máscara de atención asociada a la oración objetivo
        }

    def load_data(self, file_path):
        # Cargar datos desde el archivo CSV usando pandas
        return pd.read_csv(file_path)

# Crear instancia del conjunto de datos
train_dataset = PawXDataset(file_path=ruta_csv_train, tokenizer=tokenizer, max_length=100, target_max_length=100)
# test_dataset = PawXDataset(file_path=ruta_csv_test, tokenizer=tokenizer, max_length=100, target_max_length=100)

In [ ]:
# Crear el modelo basado en PEGASUS para parafraseo
class ParafraseadorPegasus(pl.LightningModule):
    def __init__(self, pegasus_model, tokenizer, train_dataset, batch_size=10, max_length=100):
        super(ParafraseadorPegasus, self).__init__()
        self.pegasus_model = pegasus_model
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.batch_size = batch_size
        self.max_length = max_length

    # cargar los datos durante el entrenamiento.
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def forward(self, input_ids, max_length=100):
        # define la pasada hacia adelante del modelo
        output_ids = self.pegasus_model.generate(
            input_ids, # identificadores numéricos de tokens de la oración de entrada
            max_length=max_length, # longitud máxima de la secuencia generada
            num_beams=5, # controla el número de hipótesis que el modelo considera al generar la secuencia
            length_penalty=0.8, # Es un factor que penaliza (si es menor que 1) o premia (si es mayor que 1) las secuencias más cortas o largas
            no_repeat_ngram_size=2) # Este parámetro evita que el modelo repita ciertos n-gramas en la generación
        return output_ids # contiene los identificadores de tokens de la oración parafraseada generada por el modelo

    # Paso de entrenamiento
    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        target_ids = batch['target_ids']

        loss = self.compute_loss(input_ids, target_ids) # Calcula la pérdida para el lote actual
        self.log('train_loss', loss) # Registra la pérdida en el registro para su posterior inspección y seguimiento
        return loss # Devuelve la pérdida calculada

    def compute_loss(self, input_ids, target_ids): # calcular la pérdida durante el entrenamiento
        outputs = self.pegasus_model(input_ids, labels=target_ids) # oraciones a partir de los identificadores de tokens de entrada
        loss = outputs.loss # pérdida asociada con la diferencia entre las oraciones generadas y las oraciones objetivo reales
        return loss

    def configure_optimizers(self): # configurar y devolver el optimizador que se utilizará durante el entrenamiento
        optimizer = torch.optim.AdamW(
            self.parameters(), # Se están optimizando todos los parámetros del modelo y devuelve un generador que proporciona todos los parámetros entrenables
            lr=2e-5) # Este es el valor de la tasa de aprendizaje
        return optimizer # devuelve el optimizador configurado

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids'] # extraen los identificadores de tokens de entrada
        target_ids = batch['target_ids'] # extraen los identificadores de tokens objetivo

        val_loss = self.compute_loss(input_ids, target_ids) # Se calcula la pérdida de validación
        # Es útil para realizar un seguimiento del progreso del modelo durante el entrenamiento
        self.log('val_loss', val_loss) # La pérdida de validación se registra
        return val_loss

In [ ]:
# Crear instancia del modelo
model = ParafraseadorPegasus(pegasus_model, tokenizer, train_dataset, batch_size=10, max_length=100)

In [ ]:
# # Uso de PyTorch Lightning
# logger = TensorBoardLogger("lightning_logs", name="paraphraser")
# trainer = pl.Trainer(logger=logger, max_epochs=1)
# trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                            | Params
------------------------------------------------------------------
0 | pegasus_model | PegasusForConditionalGeneration | 570 M 
------------------------------------------------------------------
568 M     Trainable params
2.1 M     Non-trainable params
570 M     Total params
2,283.188 Total estimated mode

Training: |          | 0/? [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (0) is identical to the `bos_token_id` (0), `eos_token_id` (1), or the `sep_token_id` (None), and your input is not padded.


ValueError: ignored